## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [ ]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
train, test = keras.datasets.cifar10.load_data()

In [ ]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [ ]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [ ]:
from keras.layers import BatchNormalization
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units,
                                   activation='relu',
                                   name="hidden_layer" + str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(num_units,
                                   activation='relu',
                                   name="hidden_layer" + str(i+1)))(x)
    x = BatchNormalization()(x)
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=out)
    return model

In [ ]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [ ]:
# 載入 Callbacks
"""Code Here
設定 callbacks: model checkpoint
"""
model_ckpt_best = keras.ModelCheckpoint(filepath="./best.h5", monitor="val_loss", save_best_only=True)
model_ckpt = keras.ModelCheckpoint(filepath="./checkpoint.h5", monitor="val_loss", save_best_only=False)
model_ckpt_onlyw = keras.ModelCheckpoint(filepath="./best_onlyw.h5", monitor="val_loss", save_best_only=True, save_weights_only=True)

In [ ]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)

model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[model_ckpt_best, model_ckpt, model_ckpt_onlyw]
         )

train_loss = model.history.history['train_loss']
valid_loss = model.history.history['valid_loss']
train_acc = model.history.history['train_acc']
valid_acc = model.history.history['valid_acc']

# Load back
model = keras.models.load_model("./best.h5")
loss_best, acc_best = model.evaluate(x_test, y_test)
model = keras.models.load_model("./checkpoint.h5")
loss_last, acc_last = model.evaluate(x_test, y_test)
model = build_mlp(input_shape=x_train.shape[1:])
model.load_weights("./checkpoint_onlyw.h5")
loss_best2, acc_best2 = model.evaluate(x_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""
plt.figure(figsize=(12, 9))
plt.plot(range(len(train_loss)), train_loss, '-', label='train loss')
plt.plot(range(len(valid_loss)), valid_loss, '-', label='valid loss')
plt.hlines(y=loss_best, xmin=0, xmax=len(train_loss), colors='r', linestyles='--')
plt.hlines(y=loss_last, xmin=0, xmax=len(train_loss), colors='m', linestyles='--')
plt.hlines(y=loss_best2, xmin=0, xmax=len(train_loss), colors='k', linestyles='--')
plt.title('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12, 9))
plt.plot(range(len(train_acc)), train_acc, '-', label='train accuracy')
plt.plot(range(len(valid_acc)), valid_acc, '--', label='valid accuracy')
plt.hlines(y=acc_best, xmin=0, xmax=len(train_acc), colors='r', linestyles='--')
plt.hlines(y=acc_last, xmin=0, xmax=len(train_acc), colors='m', linestyles='--')
plt.hlines(y=acc_best2, xmin=0, xmax=len(train_acc), colors='k', linestyles='--')
plt.title('Accuracy')
plt.legend()
plt.show()